In [10]:
#Model 4

import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.utils import resample
import numpy as np

def run_multiple_mediation_analyses_per_condition_glam_base(file_path, mediators, dvs, controls=None):
    results = []

    # 1. Daten laden
    data = pd.read_csv(file_path)
    data["Style"] = data["Style"].map({"Glam": 0, "Hippie": 1})

    # 2. Stelle sicher, dass alle benötigten Spalten numerisch sind
    columns = ["Style"] + mediators + dvs + (controls if controls else [])
    for col in columns:
        data[col] = pd.to_numeric(data[col], errors="coerce")

    # 3. Gültige Controls (ohne NaNs)
    valid_controls = []
    if controls:
        for ctrl in controls:
            if data[ctrl].notna().sum() > 0:
                valid_controls.append(ctrl)

    # 4. Schleife über jede Condition
    for condition, subset in data.groupby("Experiment"):
        for mediator in mediators:
            for dv in dvs:
                required_cols = ["Style", mediator, dv] + valid_controls
                df_clean = subset.dropna(subset=required_cols)
                n = len(df_clean)
                if n < 10:
                    continue  # zu wenige Daten

                control_str = " + " + " + ".join(valid_controls) if valid_controls else ""

                try:
                    # Pfad a
                    a_model = ols(f"{mediator} ~ Style{control_str}", data=df_clean).fit()
                    a = a_model.params["Style"]
                    p_a = a_model.pvalues["Style"]

                    # Pfad b & c'
                    b_model = ols(f"{dv} ~ Style + {mediator}{control_str}", data=df_clean).fit()
                    b = b_model.params[mediator]
                    p_b = b_model.pvalues[mediator]
                    c_prime = b_model.params["Style"]
                    p_c_prime = b_model.pvalues["Style"]

                    # Pfad c
                    c_model = ols(f"{dv} ~ Style{control_str}", data=df_clean).fit()
                    c = c_model.params["Style"]
                    p_c = c_model.pvalues["Style"]

                    ab = a * b

                    # Bootstrap für indirekten Effekt
                    ab_samples = []
                    for _ in range(1000):
                        sample = resample(df_clean)
                        try:
                            a_s = ols(f"{mediator} ~ Style{control_str}", data=sample).fit().params["Style"]
                            b_s = ols(f"{dv} ~ Style + {mediator}{control_str}", data=sample).fit().params[mediator]
                            ab_samples.append(a_s * b_s)
                        except:
                            continue

                    ab_ci = np.percentile(ab_samples, [2.5, 97.5])
                    p_ab = (np.sum(np.array(ab_samples) < 0) if ab > 0 else np.sum(np.array(ab_samples) > 0)) / len(ab_samples) * 2

                    # Ergebnisse speichern
                    results.append({
                        "Condition": condition,
                        "Mediator": mediator,
                        "DV": dv,
                        "n": n,
                        "a": a, "p-value a": p_a,
                        "b": b, "p-value b": p_b,
                        "c'": c_prime, "p-value c'": p_c_prime,
                        "c": c, "p-value c": p_c,
                        "a*b": ab, "p-value a*b": p_ab,
                        "BootLLCI": ab_ci[0],
                        "BootULCI": ab_ci[1]
                    })
                except Exception as e:
                    print(f"Fehler bei {condition}, {mediator}, {dv}: {e}")
                    continue

    return pd.DataFrame(results)


def color_pval(val):
    try:
        val = float(val)
    except:
        return ''
    
    if val < 0.001:
        return 'background-color: #008a00'   # strong dark green
    elif val < 0.01:
        return 'background-color: #2e7d32'   # dark green
    elif val < 0.05:
        return 'background-color: #7cbf7c'   # darker green (was too light before)
    elif val < 0.1:
        return 'background-color: #d4a017'   # dark mustard yellow
    elif val < 0.2:
        return 'background-color: #d28c8c'   # muted dark red
    else:
        return 'background-color: #9e9e9e'   # clearly darker grey


# Beispiel JewelryStore
results_jewelry = run_multiple_mediation_analyses_per_condition_glam_base(
    file_path='../Merging/Results/master_with_dv_means_recoded.csv',
    mediators=["Social_Status_Old_1_3", "Social_Status_New_1_7", 'Egoism', 'Altruism', 'Environmental_Concern'],
    dvs=['DonationMoney'],
    controls=['Age', 'DeutschkenntnisseGutSchlecht', 'GeldProMonatKleidung','GeldProMonat', 'IdentificationOutfit', 'LikingOutfit']
)

# Ergebnis visualisieren
pval_cols = [c for c in results_jewelry.columns if "p-value" in c]
styled = results_jewelry.style.applymap(color_pval, subset=pval_cols)
display(styled)


/var/folders/14/q6smmykd721g4xgkp2my8d6c0000gn/T/ipykernel_94466/1623661643.py:125: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled = results_jewelry.style.applymap(color_pval, subset=pval_cols)


,Condition,Mediator,DV,n,a,p-value a,b,p-value b,c',p-value c',c,p-value c,a*b,p-value a*b,BootLLCI,BootULCI
0,21_BeachBar_Trash_Donation_Money,Social_Status_Old_1_3,DonationMoney,96,-0.259486,0.218422,-6.673368,0.087891,-14.145876,0.068127,-12.414230,0.109452,1.731646,0.308000,-1.325954,5.932794
1,21_BeachBar_Trash_Donation_Money,Social_Status_New_1_7,DonationMoney,96,-0.427693,0.028269,-6.464280,0.130617,-15.178958,0.055866,-12.414230,0.109452,2.764728,0.232000,-1.610788,8.208946
2,21_BeachBar_Trash_Donation_Money,Egoism,DonationMoney,96,-0.391128,0.016033,-13.012195,0.010510,-17.503670,0.025199,-12.414230,0.109452,5.089440,0.032000,0.196422,12.895952
3,21_BeachBar_Trash_Donation_Money,Altruism,DonationMoney,96,0.228810,0.229969,6.858473,0.113162,-13.983515,0.071822,-12.414230,0.109452,1.569286,0.288000,-1.022321,6.385174
4,21_BeachBar_Trash_Donation_Money,Environmental_Concern,DonationMoney,96,0.114557,0.567807,5.699416,0.165434,-13.067141,0.091182,-12.414230,0.109452,0.652911,0.642000,-2.304828,4.248690
5,21_BeachBar_Trash_Donation_SpendentafelMoney,Social_Status_Old_1_3,DonationMoney,62,-0.376058,0.084919,3.059063,0.683476,-29.985068,0.016252,-31.135452,0.009979,-1.150384,0.666000,-7.985948,4.795582
6,21_BeachBar_Trash_Donation_SpendentafelMoney,Social_Status_New_1_7,DonationMoney,62,-0.331076,0.111522,3.419920,0.663285,-30.003200,0.015762,-31.135452,0.009979,-1.132252,0.678000,-8.068488,4.859546
7,21_BeachBar_Trash_Donation_SpendentafelMoney,Egoism,DonationMoney,62,-0.217762,0.264930,-1.498230,0.857122,-31.461710,0.010764,-31.135452,0.009979,0.326258,1.000000,-6.191514,5.341437
8,21_BeachBar_Trash_Donation_SpendentafelMoney,Altruism,DonationMoney,62,0.181562,0.457721,-1.617625,0.807134,-30.841754,0.011775,-31.135452,0.009979,-0.293699,0.884000,-4.659017,2.933303
9,21_BeachBar_Trash_Donation_SpendentafelMoney,Environmental_Concern,DonationMoney,62,0.243170,0.293228,-1.975836,0.778358,-30.654988,0.012678,-31.135452,0.009979,-0.480464,0.872000,-6.537596,5.284005


In [11]:
#Model 4

import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.utils import resample
import numpy as np

def run_multiple_mediation_analyses_per_condition_hippie_base(file_path, mediators, dvs, controls=None):
    results = []

    # 1. Daten laden
    data = pd.read_csv(file_path)
    data["Style"] = data["Style"].map({"Glam": 1, "Hippie": 0})

    # 2. Stelle sicher, dass alle benötigten Spalten numerisch sind
    columns = ["Style"] + mediators + dvs + (controls if controls else [])
    for col in columns:
        data[col] = pd.to_numeric(data[col], errors="coerce")

    # 3. Gültige Controls (ohne NaNs)
    valid_controls = []
    if controls:
        for ctrl in controls:
            if data[ctrl].notna().sum() > 0:
                valid_controls.append(ctrl)

    # 4. Schleife über jede Condition
    for condition, subset in data.groupby("Experiment"):
        for mediator in mediators:
            for dv in dvs:
                required_cols = ["Style", mediator, dv] + valid_controls
                df_clean = subset.dropna(subset=required_cols)
                n = len(df_clean)
                if n < 10:
                    continue  # zu wenige Daten

                control_str = " + " + " + ".join(valid_controls) if valid_controls else ""

                try:
                    # Pfad a
                    a_model = ols(f"{mediator} ~ Style{control_str}", data=df_clean).fit()
                    a = a_model.params["Style"]
                    p_a = a_model.pvalues["Style"]

                    # Pfad b & c'
                    b_model = ols(f"{dv} ~ Style + {mediator}{control_str}", data=df_clean).fit()
                    b = b_model.params[mediator]
                    p_b = b_model.pvalues[mediator]
                    c_prime = b_model.params["Style"]
                    p_c_prime = b_model.pvalues["Style"]

                    # Pfad c
                    c_model = ols(f"{dv} ~ Style{control_str}", data=df_clean).fit()
                    c = c_model.params["Style"]
                    p_c = c_model.pvalues["Style"]

                    ab = a * b

                    # Bootstrap für indirekten Effekt
                    ab_samples = []
                    for _ in range(1000):
                        sample = resample(df_clean)
                        try:
                            a_s = ols(f"{mediator} ~ Style{control_str}", data=sample).fit().params["Style"]
                            b_s = ols(f"{dv} ~ Style + {mediator}{control_str}", data=sample).fit().params[mediator]
                            ab_samples.append(a_s * b_s)
                        except:
                            continue

                    ab_ci = np.percentile(ab_samples, [2.5, 97.5])
                    p_ab = (np.sum(np.array(ab_samples) < 0) if ab > 0 else np.sum(np.array(ab_samples) > 0)) / len(ab_samples) * 2

                    # Ergebnisse speichern
                    results.append({
                        "Condition": condition,
                        "Mediator": mediator,
                        "DV": dv,
                        "n": n,
                        "a": a, "p-value a": p_a,
                        "b": b, "p-value b": p_b,
                        "c'": c_prime, "p-value c'": p_c_prime,
                        "c": c, "p-value c": p_c,
                        "a*b": ab, "p-value a*b": p_ab,
                        "BootLLCI": ab_ci[0],
                        "BootULCI": ab_ci[1]
                    })
                except Exception as e:
                    print(f"Fehler bei {condition}, {mediator}, {dv}: {e}")
                    continue

    return pd.DataFrame(results)


def color_pval(val):
    try:
        val = float(val)
    except:
        return ''
    
    if val < 0.001:
        return 'background-color: #008a00'   # strong dark green
    elif val < 0.01:
        return 'background-color: #2e7d32'   # dark green
    elif val < 0.05:
        return 'background-color: #7cbf7c'   # darker green (was too light before)
    elif val < 0.1:
        return 'background-color: #d4a017'   # dark mustard yellow
    elif val < 0.2:
        return 'background-color: #d28c8c'   # muted dark red
    else:
        return 'background-color: #9e9e9e'   # clearly darker grey


# Beispiel JewelryStore
results_jewelry = run_multiple_mediation_analyses_per_condition_hippie_base(
    file_path='../Merging/Results/master_with_dv_means_recoded.csv',
    mediators=["Social_Status_Old_1_3", "Social_Status_New_1_7", 'Egoism', 'Altruism', 'Environmental_Concern'],
    dvs=['DonationTime'],
    controls=['Age', 'DeutschkenntnisseGutSchlecht', 'GeldProMonatKleidung','GeldProMonat', 'IdentificationOutfit', 'LikingOutfit']
)

# Ergebnis visualisieren
pval_cols = [c for c in results_jewelry.columns if "p-value" in c]
styled = results_jewelry.style.applymap(color_pval, subset=pval_cols)
display(styled)


/var/folders/14/q6smmykd721g4xgkp2my8d6c0000gn/T/ipykernel_94466/2981577437.py:125: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled = results_jewelry.style.applymap(color_pval, subset=pval_cols)


,Condition,Mediator,DV,n,a,p-value a,b,p-value b,c',p-value c',c,p-value c,a*b,p-value a*b,BootLLCI,BootULCI
0,21_BeachBar_Trash_Donation_SpendentafelTime,Social_Status_Old_1_3,DonationTime,63,0.334024,0.191663,3.823699,0.540479,-7.008229,0.555555,-5.731023,0.622219,1.277207,0.608000,-3.634385,8.712967
1,21_BeachBar_Trash_Donation_SpendentafelTime,Social_Status_New_1_7,DonationTime,63,0.491579,0.056300,-7.273199,0.243198,-2.155670,0.857127,-5.731023,0.622219,-3.575353,0.318000,-12.811840,2.802086
2,21_BeachBar_Trash_Donation_SpendentafelTime,Egoism,DonationTime,63,0.255991,0.205562,-22.465849,0.003116,0.020026,0.998543,-5.731023,0.622219,-5.751049,0.214000,-16.353470,3.101129
3,21_BeachBar_Trash_Donation_SpendentafelTime,Altruism,DonationTime,63,-0.340828,0.152769,-5.359997,0.424165,-7.557859,0.525357,-5.731023,0.622219,1.826836,0.484000,-2.963481,10.533514
4,21_BeachBar_Trash_Donation_SpendentafelTime,Environmental_Concern,DonationTime,63,-0.367494,0.111036,7.786586,0.261032,-2.869503,0.808909,-5.731023,0.622219,-2.861520,0.318000,-9.321770,2.092631
5,21_BeachBar_Trash_Donation_Time,Social_Status_Old_1_3,DonationTime,69,0.029208,0.881190,-0.284734,0.963661,-4.789008,0.614593,-4.797325,0.610958,-0.008317,0.946000,-2.747878,2.465024
6,21_BeachBar_Trash_Donation_Time,Social_Status_New_1_7,DonationTime,69,0.044908,0.801847,-4.514544,0.507609,-4.594584,0.627862,-4.797325,0.610958,-0.202741,0.988000,-3.656336,3.243813
7,21_BeachBar_Trash_Donation_Time,Egoism,DonationTime,69,-0.080339,0.625479,-14.138113,0.053158,-5.933161,0.520775,-4.797325,0.610958,1.135836,0.698000,-4.701613,6.972883
8,21_BeachBar_Trash_Donation_Time,Altruism,DonationTime,69,-0.306893,0.144710,6.454018,0.267869,-2.816634,0.768559,-4.797325,0.610958,-1.980691,0.366000,-8.463588,1.815010
9,21_BeachBar_Trash_Donation_Time,Environmental_Concern,DonationTime,69,-0.088023,0.718954,10.462823,0.032778,-3.876356,0.672127,-4.797325,0.610958,-0.920968,0.740000,-6.764629,5.837921


In [12]:
#Model 4

import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.utils import resample
import numpy as np

# Beispiel JewelryStore
results_jewelry = run_multiple_mediation_analyses_per_condition_hippie_base(
    file_path='../Merging/Results/master_with_dv_means_recoded.csv',
    mediators=["Social_Status_Old_1_3", "Social_Status_New_1_7", 'Egoism', 'Altruism', 'Environmental_Concern'],
    dvs=['trashCount'],
    controls=['Age', 'DeutschkenntnisseGutSchlecht', 'GeldProMonatKleidung','GeldProMonat', 'IdentificationOutfit', 'LikingOutfit']
)

# Ergebnis visualisieren
pval_cols = [c for c in results_jewelry.columns if "p-value" in c]
styled = results_jewelry.style.applymap(color_pval, subset=pval_cols)
display(styled)


/var/folders/14/q6smmykd721g4xgkp2my8d6c0000gn/T/ipykernel_94466/3005582378.py:19: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled = results_jewelry.style.applymap(color_pval, subset=pval_cols)


,Condition,Mediator,DV,n,a,p-value a,b,p-value b,c',p-value c',c,p-value c,a*b,p-value a*b,BootLLCI,BootULCI
0,21_BeachBar_Trash_Donation_Money,Social_Status_Old_1_3,trashCount,96,0.259486,0.218422,-1.745042,0.628887,-3.564314,0.618243,-4.017128,0.569415,-0.452814,0.712000,-2.872852,2.504841
1,21_BeachBar_Trash_Donation_Money,Social_Status_New_1_7,trashCount,96,0.427693,0.028269,-1.023786,0.795159,-3.579262,0.623686,-4.017128,0.569415,-0.437866,0.786000,-3.869783,3.460150
2,21_BeachBar_Trash_Donation_Money,Egoism,trashCount,96,0.391128,0.016033,-0.262781,0.955867,-3.914347,0.593840,-4.017128,0.569415,-0.102781,0.902000,-4.492024,3.935814
3,21_BeachBar_Trash_Donation_Money,Altruism,trashCount,96,-0.228810,0.229969,0.533458,0.893791,-3.895068,0.586399,-4.017128,0.569415,-0.122060,0.936000,-3.230606,2.485816
4,21_BeachBar_Trash_Donation_Money,Environmental_Concern,trashCount,96,-0.114557,0.567807,0.499010,0.895137,-3.959963,0.577776,-4.017128,0.569415,-0.057165,1.082000,-1.584281,2.405178
5,21_BeachBar_Trash_Donation_SpendentafelMoney,Social_Status_Old_1_3,trashCount,62,0.376058,0.084919,0.621590,0.895382,-12.870628,0.096896,-12.636874,0.090840,0.233753,0.906000,-3.486190,3.553370
6,21_BeachBar_Trash_Donation_SpendentafelMoney,Social_Status_New_1_7,trashCount,62,0.331076,0.111522,-5.340775,0.277903,-10.868674,0.153257,-12.636874,0.090840,-1.768201,0.368000,-6.661437,1.611083
7,21_BeachBar_Trash_Donation_SpendentafelMoney,Egoism,trashCount,62,0.217762,0.264930,-2.953544,0.572410,-11.993704,0.114418,-12.636874,0.090840,-0.643171,0.698000,-5.481849,2.242388
8,21_BeachBar_Trash_Donation_SpendentafelMoney,Altruism,trashCount,62,-0.181562,0.457721,2.715242,0.514404,-12.143891,0.107477,-12.636874,0.090840,-0.492984,0.642000,-4.891847,1.220085
9,21_BeachBar_Trash_Donation_SpendentafelMoney,Environmental_Concern,trashCount,62,-0.243170,0.293228,5.071198,0.248385,-11.403710,0.128800,-12.636874,0.090840,-1.233164,0.466000,-5.923188,2.020875
